<a href="https://colab.research.google.com/github/shilpa2503/Machine-Learning/blob/main/MLP_Adult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import numpy as np
import torch
import os
import torch
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
header = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 
                  'capital-loss', 'hours-per-week', 'native-country', 'salary']
try:
  df_temp = pd.read_csv("adults.csv",index_col=False, skipinitialspace=True,header=None,names=header)
except:
  df_temp = pd.read_csv("https://raw.githubusercontent.com/aliakbarbadri/mlp-classifier-adult-dataset/master/adults.csv", index_col=False, skipinitialspace=True,header=None,names=header)

In [ ]:
df = df_temp

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
dtype: object

In [ ]:
df = df.replace('?', np.nan)
df[pd.isnull(df).any(axis=1)].shape

(3620, 15)

In [ ]:
print(df.shape)

(48842, 15)


In [ ]:
categorical_columns = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country']
label_column = ['salary']

In [ ]:
def show_unique_values(columns):
  for column in columns:
    uniq = df[column].unique().tolist()
    print(column+ " has "+ str(len(uniq)) +" values" + " : " + str(uniq))
show_unique_values(categorical_columns)
show_unique_values(label_column)

workclass has 9 values : ['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov', 'Local-gov', nan, 'Self-emp-inc', 'Without-pay', 'Never-worked']
education has 16 values : ['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school', '5th-6th', '10th', '1st-4th', 'Preschool', '12th']
marital-status has 7 values : ['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated', 'Married-AF-spouse', 'Widowed']
occupation has 15 values : ['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners', 'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair', 'Transport-moving', 'Farming-fishing', 'Machine-op-inspct', 'Tech-support', nan, 'Protective-serv', 'Armed-Forces', 'Priv-house-serv']
relationship has 6 values : ['Not-in-family', 'Husband', 'Wife', 'Own-child', 'Unmarried', 'Other-relative']
race has 5 values : ['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other']
sex has 

In [ ]:
def convert_to_int(columns):
  for column in columns:
    unique_values = df[column].unique().tolist()
    dic = {}
    for indx, val in enumerate(unique_values):
      dic[val]=indx
    df[column] = df[column].map(dic).astype(int)
    print(column + " done!")
convert_to_int(label_column)
show_unique_values(label_column)

salary done!
salary has 2 values : [0, 1]


In [ ]:
def convert_to_onehot(data,columns):
  dummies = pd.get_dummies(data[columns])
  data = data.drop(columns, axis=1)
  data = pd.concat([data, dummies], axis=1)
  return data

In [ ]:
df = convert_to_onehot(df,categorical_columns)

In [ ]:
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.shape

(48842, 106)

In [ ]:
normalize_columns = ['age', 'fnlwgt', 'capital-gain','capital-loss','hours-per-week']

In [ ]:
def show_values(columns):
  for column in columns:
    max_val = df[column].max()
    min_val = df[column].min()
    mean_val = df[column].mean()
    var_val = df[column].var()
    print(column +': values=['+str(min_val)+','+str(max_val)+'] , mean='+str(mean_val)+' , var='+str(var_val))

In [ ]:
show_values(normalize_columns)

age: values=[17,90] , mean=38.64358543876172 , var=187.97808266247543
fnlwgt: values=[12285,1490400] , mean=189664.13459727284 , var=11152210185.574848
capital-gain: values=[0,99999] , mean=1079.0676262233324 , var=55532588.035659194
capital-loss: values=[0,4356] , mean=87.50231358257237 , var=162412.66903295522
hours-per-week: values=[1,99] , mean=40.422382375824085 , var=153.5478850061782


In [ ]:
df_1 = df

In [ ]:
def normalize(columns):
  scaler = preprocessing.StandardScaler()
  df[columns] = scaler.fit_transform(df[columns])
normalize(normalize_columns)
show_values(normalize_columns)

age: values=[-1.578628884921691,3.7458081832961794] , mean=2.2752730001408557e-16 , var=1.000020474601257
fnlwgt: values=[-1.6796798205441077,12.317231053897183] , mean=-5.644539156359668e-17 , var=1.0000204746012575
capital-gain: values=[-0.144803531037397,13.274381650977991] , mean=1.0183446931576719e-17 , var=1.000020474601257
capital-loss: values=[-0.21712709919583073,10.591794463826847] , mean=-9.892491304960242e-18 , var=1.0000204746012573
hours-per-week: values=[-3.1814521274018306,4.727311617850181] , mean=4.5680033378787e-17 , var=1.000020474601257


In [ ]:
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.025996,-1.061979,13,0.146932,-0.217127,-0.034087,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.828308,-1.007104,13,-0.144804,-0.217127,-2.213032,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.046942,0.246034,9,-0.144804,-0.217127,-0.034087,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1.047121,0.426663,7,-0.144804,-0.217127,-0.034087,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-0.776316,1.408530,13,-0.144804,-0.217127,-0.034087,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_x = df.drop('salary', axis=1)

In [ ]:
df_y = df['salary']

In [ ]:
df_y = df_y.astype(int)

In [ ]:
X = df_x.to_numpy()
Y = df_y.to_numpy()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
train_dataset = DataLoader(
      TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long()))

In [ ]:
test_dataset = DataLoader(
      TensorDataset(torch.from_numpy(x_val).float(), torch.from_numpy(y_val).long()))

In [ ]:
dataset_train = train_dataset
dataset_test = test_dataset

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import math

class MLP(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(MLP, self).__init__()
        self.layer_input = nn.Linear(dim_in, dim_hidden)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout()
        self.layer_hidden = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = self.layer_input(x)
        # x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_hidden(x)
        return x

In [ ]:
num_classes = 2
len_in = 1
dataset_train = dataset_train.dataset
dataset_test = dataset_test.dataset
data_size = dataset_train[0][0].shape
for x in data_size:
    len_in *= x
net = MLP(dim_in=len_in, dim_hidden=256, dim_out=num_classes)

In [ ]:
def train(net):
  loss_func = nn.CrossEntropyLoss()
  batch_size = 64
  ldr_train = DataLoader((dataset_train), batch_size=batch_size, shuffle=True)

  net.train()
    # train and update

  optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

  epoch_loss = []
  for iter in range(25):
      batch_loss = []
      for batch_idx, (images, labels) in enumerate(ldr_train):
          net.zero_grad()
          log_probs = net(images)

          loss = loss_func(log_probs, labels)

          loss.backward()
          optimizer.step()

          batch_loss.append(loss.item())

      epoch_loss.append(sum(batch_loss) / len(batch_loss))

  return net.state_dict(), sum(epoch_loss) / len(epoch_loss)

In [ ]:
w, loss = train(net)

In [ ]:
net.load_state_dict(w)

<All keys matched successfully>

In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
def test_img(net, datatest):
    net.eval()
    # testing
    test_loss = 0
    correct = 0
    data_loader = DataLoader(datatest, batch_size=64)
    l = len(data_loader)
    for idx, (data, target) in enumerate(data_loader):
        log_probs = net(data)
        # sum up batch loss
        test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
        # get the index of the max log-probability
        y_pred = log_probs.data.max(1, keepdim=True)[1]
        correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    test_loss /= len(data_loader.dataset)
    accuracy = 100.00 * correct / len(data_loader.dataset)

    return accuracy, test_loss

In [ ]:
acc_train, loss_train_ = test_img(net, dataset_train)
acc_test, loss_test = test_img(net, dataset_test)

In [ ]:
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))

Training accuracy: 85.73
Testing accuracy: 85.06
